In [6]:
from metaflow import Flow, namespace
namespace(None)
post_run =  next(Flow('HNSentimentAnalyzeComments').runs('analyze_this'))
sentiments = {}
for task in post_run['analyze_comments']:
    if 'post_sentiment' in task:
        sentiments.update((int(post_id), score) for post_id, (score, _) in task['post_sentiment'].data.items())
len(sentiments)

65743

In [32]:
from collections import Counter
def all_topics(filterset=None):
    for post_id, (topic_lst, _) in topics.items():
        if filterset is None or post_id in filterset:
            for topic in topic_lst:
                yield topic.strip()

In [14]:
from metaflow import Flow, namespace
namespace(None)
post_run =  next(Flow('HNSentimentAnalyzePosts').runs('analyze_this'))
topics = {int(post_id): topics for post_id, topics in post_run['end'].task['post_tags'].data.items()}
post_run

Run('HNSentimentAnalyzePosts/argo-hnsentiment.user.tuulosgmail.com.hnsentimentanalyzeposts-rhdls')

In [29]:
from collections import defaultdict
topic_sentiment = defaultdict(list)
for post_id, (topic_lst, _) in topics.items():
    score = sentiments.get(post_id)
    if score is not None:
        for topic in topic_lst:
            topic_sentiment[topic].append(score)
median_sentiments = {}
for topic, scores in topic_sentiment.items():
    scores.sort()
    median_sentiments[topic] = scores[len(scores) // 2]

In [35]:
freqs = Counter(all_topics())
mese = {t: c for t, c in median_sentiments.items() if freqs[t] > 10}
list(sorted(mese.items(), key=lambda x: x[1], reverse=True))

[('Pathfinding', 9),
 ('ZX Spectrum', 9),
 ('Sequencer', 9),
 ('Flexbox', 9),
 ('3D Animation', 9),
 ('Math History', 9),
 ('PICO-8', 9),
 ('World', 9),
 ('Indie', 9),
 ('Bevy', 9),
 ('IEEE Spectrum', 9),
 ('Academic Resources', 9),
 ('Math', 8),
 ('Binary', 8),
 ('Number theory', 8),
 ('Learning', 8),
 ('Arithmetic', 8),
 ('Profiling', 8),
 ('Optimization', 8),
 ('Python', 8),
 ('Memory', 8),
 ('Performance Benchmarking', 8),
 ('Performance Improvement', 8),
 ('Programming', 8),
 ('Porting', 8),
 ('Language Evolution', 8),
 ('DIY Project', 8),
 ('GitHub', 8),
 ('Scripting', 8),
 ('Coding', 8),
 ('SoftwareDevelopment', 8),
 ('OpenSource', 8),
 ('Nature', 8),
 ('Solitude', 8),
 ('Childhood', 8),
 ('Microscopy', 8),
 ('Mathematics', 8),
 ('PostgreSQL', 8),
 ('Configuration', 8),
 ('Database', 8),
 ('SQL', 8),
 ('Tools', 8),
 ('Utilities', 8),
 ('Database Management', 8),
 ('Learning Theory', 8),
 ('Winamp', 8),
 ('Plugins', 8),
 ('Music Player', 8),
 ('Screen Sharing', 8),
 ('Image Proce

In [54]:
divisive = []
for topic, scores in topic_sentiment.items():
    if len(scores) > 50:
        num_high = sum(1 for s in scores if s >= 7)
        num_low = sum(1 for s in scores if s <= 4)
        z = num_high / (num_high + num_low)
        if (num_high + num_low) / len(scores) > 0.7 and 0.4 < z < 0.6:
            divisive.append((z, topic))

divisive.sort()
divisive

[(0.4035608308605341, 'Tech Industry'),
 (0.40384615384615385, 'Reddit'),
 (0.4050632911392405, 'Fintech'),
 (0.4107142857142857,
  "Please provide the article text, and I'll be happy to assist you with assigning relevant tags."),
 (0.411829134720701, 'Privacy'),
 (0.41509433962264153, 'Piracy'),
 (0.421875, 'Poverty'),
 (0.4222222222222222, 'Apps'),
 (0.423728813559322, 'Investing'),
 (0.4365079365079365, 'Law'),
 (0.4444444444444444, 'Mozilla'),
 (0.4473684210526316, 'Covid-19'),
 (0.45255474452554745, 'Online Security'),
 (0.45714285714285713, 'Healthcare'),
 (0.4594594594594595, 'Accountability'),
 (0.46511627906976744, 'Human Resources'),
 (0.46794871794871795, 'Europe'),
 (0.4772727272727273, 'Wikipedia'),
 (0.4835262689225289, 'Cybersecurity'),
 (0.4838709677419355, 'Autonomous Vehicles'),
 (0.4900990099009901, 'Data Protection'),
 (0.49333333333333335, 'Technical Issue'),
 (0.5, 'Google'),
 (0.5, 'Monetization'),
 (0.5, 'Online Services'),
 (0.5027322404371585, 'E-commerce'),
 

In [58]:
import json
len(json.dumps(median_sentiments)) / 1024**2

2.213017463684082

In [1]:
import duckdb
post_times = dict(duckdb.query("select id, time from 'story.parquet' where to_timestamp(time) > timestamp '2020-01-01'").execute().fetchall())

In [16]:
from datetime import datetime
import pandas as pd
from collections import defaultdict

CUTOFF = datetime(2023, 6, 1).timestamp()

def timeseries():
    days = defaultdict(list)
    for post_id, tstamp in post_times.items():
        if tstamp < CUTOFF:  
            if post_id in sentiments:
                day = datetime.fromtimestamp(tstamp).replace(hour=0, minute=0, second=0)
                days[day].append(sentiments[post_id])
    dfdata = []
    for day, series in days.items():
        series.sort()
        median = sum(series) / len(series)
        dfdata.append({'date': day, 'mood': median})  
    return pd.DataFrame(dfdata)

mood_df = timeseries()
mood_df

,date,mood
0,2022-05-13,6.653061
1,2021-08-19,5.736842
2,2020-03-08,6.479167
3,2020-08-14,6.290323
4,2021-06-19,5.847826
...,...,...
1242,2022-04-05,6.581395
1243,2021-05-23,6.487805
1244,2022-04-27,6.255319
1245,2023-01-13,5.745098


In [24]:

import altair as alt

alt.renderers.enable('html')

from altair_theme import THEME
alt.themes.register('ob', lambda: THEME)
alt.themes.enable('ob')

base = alt.Chart(mood_df).properties(width=1000, height=700)
chart = base.mark_point().encode(x="date", y="mood")
chart + chart.transform_loess('date', 'mood').mark_line().encode(color=alt.value('#fc6603'))

alt.LayerChart(...)

In [48]:
df = pd.DataFrame({'sentiment': list(sentiments.values())})

alt.data_transformers.disable_max_rows()

alt.Chart(df).mark_bar().encode(
    x=alt.X("sentiment:O", bin=True, title="Sentiment score"),
    y=alt.Y('count()', title="Number of posts"),
).properties(width=600, height=300).configure(axisBottom=alt.AxisConfig(labelFontSize=16, titleFontSize=16, labelExpr='datum.label[0]', labelAngle=0),
                                              axisLeft=alt.AxisConfig(labelFontSize=14, titleFontSize=16))


alt.Chart(...)